# DeepSeek Agent 实战：小红书爆款文案生成助手

本 Notebook 将指导您如何使用 DeepSeek LLM 构建一个能够生成小红书爆款文案的智能 Agent。我们将从需求拆解开始，逐步定义 Agent 的系统提示词 (System Prompt)、外部工具 (Tools)，并实现其核心的工作流程，最终生成符合小红书平台特点的文案。

## 1. 环境准备与DeepSeek API配置

In [1]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

## 2. 需求拆解与Agent任务规划

#### 用户痛点与核心需求：
*   **效率低下：** 人工创作周期长，难以满足高频发布需求。
*   **创意瓶颈：** 难以持续产出新颖、吸引人的爆款创意。
*   **趋势捕捉难：** 实时流行元素难以快速融入文案。
*   **平台特性把握：** 小红书特有风格（标题、正文、标签、表情）难以精准复制。

#### “爆款”文案的特征：
*   **强吸引力标题：** 制造好奇、痛点共鸣、利益点清晰。
*   **沉浸式正文：** 真实体验分享、细节描述、情感共鸣。
*   **精准且多样标签：** 热门话题、品牌词、产品词、垂直领域词。
*   **生动表情符号：** 增强表达力，提升活泼感。
*   **清晰的行动召唤 (CTA)。**

#### Agent 任务规划：核心工作流
1.  **用户指令接收：** 接收产品信息、主题、风格等。
2.  **信息收集 (Web Search/DB Query)：** 实时搜索行业趋势、热门话题、竞品分析、产品卖点。
3.  **内容构思与初稿生成 (LLM)：** 结合所有信息，撰写标题、正文、标签、表情符号。
4.  **风格与格式优化 (LLM)：** 根据小红书平台特点和指定风格，对文案进行润色和结构调整。
5.  **最终输出：** 呈现完整文案。

## 3. 爆款文案生成逻辑与 Prompt 设计

### 3.1 System Prompt (系统提示词)

System Prompt 是 Agent 的“大脑”和“行为准则”。它定义了 Agent 的角色、目标以及工作方式。我们将采用 `Thought-Action-Observation` (ReAct) 模式来引导 DeepSeek 的推理过程。

In [2]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

### 3.2 Tools (工具定义)

Agent 的“双手”由一系列可调用的工具组成。这些工具扩展了 LLM 的能力，使其能够获取实时信息、查询数据库或执行特定操作。在这里，我们定义了三个核心工具：

*   `search_web`: 用于搜索互联网上的实时信息，如最新趋势、用户评价等。
*   `query_product_database`: 用于查询产品数据库，获取产品的详细卖点和特点。**此工具为模拟**。
*   `generate_emoji`: 用于根据文案内容生成恰当的表情符号。**此工具为模拟**。

In [3]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

### 3.3 模拟工具实现

由于我们无法直接调用真实的外部 API (如Google Search或内部产品数据库)，我们将创建一些模拟 (Mock) 工具函数来演示 Agent 的工作流程。在实际应用中，您需要将这些模拟函数替换为真实的 API 调用。

In [9]:
import random # 用于模拟生成表情
import time # 用于模拟网络延迟

def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟

    # 主题1：美妆类
    if "小红书美妆趋势" in query:
        return "近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。"
    elif "深海蓝藻保湿面膜" in query:
        return "关于深海蓝藻保湿面膜的用户评价：普遍反馈补水效果好，吸收快，对敏感肌友好。有用户提到价格略高，但效果值得。"
    
    # 主题2：护肤成分科普
    elif "玻尿酸" in query:
        return "玻尿酸是2024年最热门的护肤成分。小红书话题#玻尿酸护肤有超100万浏览量。用户反馈：补水能力强、适合所有肤质、与其他成分配合效果更佳。常见浓度为0.1%~2%。"
    elif "烟酰胺美白" in query:
        return "烟酰胺美白精华在小红书爆火，话题热度持续上升。用户评价：提亮肤色明显、使用周期短（2-4周见效）、温和不刺激。特别适合熬夜人群改善暗沉。"
    elif "视黄醇抗衰" in query:
        return "视黄醇被誉为'护肤黄金成分'，小红书笔记讨论度极高。用户心得：初期可能有刺激感，需要逐步建立耐受。坚持3个月可见细纹淡化、肤质提亮。"
    
    # 主题3：身体护理
    elif "身体护肤趋势" in query:
        return "2024年身体护肤成为新风尚，小红书话题#身体护理有800万+讨论。热门需求：颈部紧致、手部抗老、身体美白。消费者追求从头到脚的完整护理方案。"
    elif "颈部精华" in query:
        return "颈部护理已成显性需求。用户反馈颈纹困扰严重，市面上颈部专用精华热度上升。高效成分组合（视黄醇+肽+透明质酸）能在4-6周改善颈纹。"
    elif "手部护理霜" in query:
        return "手部护理霜销量暴增，消费者将其视为'第二张脸'。产品特点：快速吸收、持久保湿、改善手部暗沉和干纹。早晚使用效果最佳，按摩可促进吸收。"
    
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。"

def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5) # 模拟数据库查询延迟

    # 主题1：美妆面膜
    if "深海蓝藻保湿面膜" in product_name:
        return "深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。价格：¥128/盒。"
    
    # 主题2：精华类
    elif "烟酰胺美白精华" in product_name:
        return "烟酰胺美白精华：核心成分烟酰胺5%+VC衍生物3%+熊果苷2%，三重美白协同配方。功效：提亮肤色、淡化痘印、改善暗沉、抑制黑色素。质地轻薄易吸收，清爽不油腻，适合混合肌和油肌。规格：30ml。价格：¥99/瓶。使用周期：早晚各1次，2-4周见效。"
    elif "视黄醇抗衰精华" in product_name:
        return "视黄醇抗衰精华：含0.3%稳定型视黄醇+肽类复合物+透明质酸。功效：淡化细纹、紧致肌肤、促进胶原蛋白合成、改善肤质。质地乳霜状，易吸收。使用建议：初期每周2-3次，逐步增加至每日使用。规格：30ml。价格：¥189/瓶。"
    elif "玻尿酸补水精华" in product_name:
        return "玻尿酸补水精华：含小分子玻尿酸+中分子玻尿酸+大分子玻尿酸三重层次补水。功效：即时补水、保湿锁水、改善干纹、提升肤质。质地清爽透明，吸收快。规格：50ml。价格：¥79/瓶。适合所有肤质，尤其沙漠肌必备。"
    
    # 主题3：身体护理
    elif "颈部紧致精华" in product_name:
        return "颈部紧致精华：专为颈部肌肤设计，含视黄醇0.2%+多肽复合物+卡波姆。功效：淡化颈纹、紧致肌肤、提亮肤色、改善皮肤松弛。质地轻薄，快速吸收，不油腻。规格：30ml。价格：¥169/瓶。建议早晚使用，配合按摩效果更佳。"
    elif "手部护理霜" in product_name:
        return "手部护理霜：富含甘油、角鲨烷、维E和玫瑰果油。功效：深层滋养、快速吸收、改善手部干纹和暗沉、柔软嫩白。质地丰润但不油腻，独特香气。规格：75ml。价格：¥89/罐。早晚涂抹配合按摩，1周见效。"
    elif "身体美白乳液" in product_name:
        return "身体美白乳液：含烟酰胺3%+光甘草定+视黄醇棕榈酸酯。功效：身体美白、改善肤色不均、淡化痘印、保湿滋养。质地清爽乳液状，吸收快。规格：200ml。价格：¥129/瓶。全身使用，坚持4周肤色明显提亮。"
    
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"

def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    
    # 补水保湿相关
    if "补水" in context or "水润" in context or "保湿" in context or "玻尿酸" in context:
        return ["💦", "💧", "🌊", "✨", "💙"]
    
    # 美白提亮相关
    elif "美白" in context or "提亮" in context or "烟酰胺" in context or "闪闪" in context:
        return ["✨", "💫", "⭐", "🌟", "☀️"]
    
    # 抗衰紧致相关
    elif "抗衰" in context or "紧致" in context or "视黄醇" in context or "细纹" in context:
        return ["💪", "🎯", "✅", "🔧", "🏆"]
    
    # 舒缓修护相关
    elif "舒缓" in context or "修护" in context or "敏感" in context or "屏障" in context:
        return ["🩹", "💚", "🌿", "🛡️", "☘️"]
    
    # 惊喜热情相关
    elif "惊喜" in context or "哇塞" in context or "爱了" in context or "好用" in context:
        return ["💖", "😍", "🤩", "💯", "🔥"]
    
    # 熬夜疲惫相关
    elif "熬夜" in context or "疲惫" in context or "急救" in context:
        return ["😭", "😮‍💨", "😴", "💡", "🚨"]
    
    # 推荐好物相关
    elif "好物" in context or "推荐" in context or "必买" in context or "平价" in context:
        return ["✅", "👍", "⭐", "🛍️", "💝"]
    
    # 身体护理相关
    elif "身体" in context or "颈部" in context or "手部" in context or "全身" in context:
        return ["💅", "🧴", "🧖‍♀️", "💪", "✨"]
    
    # 成分科普相关
    elif "成分" in context or "科学" in context or "研究" in context or "专业" in context:
        return ["🔬", "📊", "💡", "🧬", "📚"]
    
    else:
        return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿", "⭐"], k=5)

# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": mock_query_product_database,
    "generate_emoji": mock_generate_emoji,
}

## 4. 实战：构建小红书文案生成 Agent

现在，我们将把 System Prompt、工具定义和模拟工具函数整合起来，构建出能够自动执行的 DeepSeek Agent 工作流。核心是 `generate_rednote` 函数，它通过一个循环来模拟 Agent 的 `Thought-Action-Observation` 过程。

In [5]:
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    
    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。
        
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """
    
    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")
    
    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]
    
    iteration_count = 0
    final_response = None
    
    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")
        
        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=TOOLS_DEFINITION, # 告知模型可用的工具
                tool_choice="auto" # 允许模型自动决定是否使用工具
            )

            response_message = response.choices[0].message
            
            # **ReAct模式：处理工具调用**
            if response_message.tool_calls: # 如果模型决定调用工具
                print("Agent: 决定调用工具...")
                messages.append(response_message) # 将工具调用信息添加到对话历史
                
                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 确保参数是合法的JSON字符串，即使工具不要求参数，也需要传递空字典
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}")
                    
                    # 查找并执行对应的模拟工具函数
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果：{tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result) # 工具结果作为字符串返回
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs) # 将工具执行结果作为 Observation 添加到对话历史
                
            # **ReAct 模式：处理最终内容**
            elif response_message.content: # 如果模型直接返回内容（通常是最终答案）
                print(f"[模型生成结果] {response_message.content}")
                
                # --- START: 添加 JSON 提取和解析逻辑 ---
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)
                
                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message) # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                        messages.append(response_message) # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break
                
        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break
    
    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

## 5. 实际测试与文案生成

现在，让我们调用我们构建的 `generate_rednote` 函数，看看它能生成什么样的爆款文案！

In [11]:
# 测试案例 1: 小红书美妆趋势
product_name_1 = "小红书美妆趋势"
tone_style_1 = "知性温柔"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：小红书美妆趋势，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '2024年小红书美妆趋势最新流行彩妆护肤'}
[Tool Call] 模拟搜索网页：2024年小红书美妆趋势最新流行彩妆护肤
Observation: 工具返回结果：近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '小红书美妆博主推荐2024彩妆护肤趋势分析'}
[Tool Call] 模拟搜索网页：小红书美妆博主推荐2024彩妆护肤趋势分析
Observation: 工具返回结果：未找到关于 '小红书美妆博主推荐2024彩妆护肤趋势分析' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。
-- Iteration 3 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '美妆趋势分析护肤彩妆流行温柔知性'}
[Tool Call] 模拟生成表情符号，上下文：美妆趋势分析护肤彩妆流行温柔知性
Observation: 工具返回结果：['✨', '🔥', '👍', '🎉', '💧']
-- Iteration 4 --
[模型生成结果] **Observation**: 我获得了5个表情符号。现在我可以创作这篇关于小红书美妆趋势的文案了。我需要创作一篇语气知性温柔、包含标题、正文、至少5个相关标签和5个表情符号的完整小红书笔记。

**Final Action**: 基于收集到的信息，我将创作一篇关于小红书美妆趋势的爆款文案。文案将采用知性温柔的语气，涵盖当前流行的美妆趋势，并提供实用的建议。

```json
{
  "title": "2024小红书美妆趋势全解析✨温柔知性风正当时",
  "body": "姐妹们好呀～今天想和大家聊聊2024年小红书上最火的美妆趋势，用温柔知性的视角带你们看懂这些流行密码✨\n\n

In [12]:
# 测试案例 2: 玻尿酸
product_name_2 = "玻尿酸"
tone_style_2 = "活泼搞怪"
result_2 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 2 ---")
print(result_2)


🚀 启动小红书文案生成助手，产品：玻尿酸，风格：活泼搞怪

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '玻尿酸'}
[Tool Call] 模拟查询产品数据库：玻尿酸
Observation: 工具返回结果：产品数据库中未找到关于 '玻尿酸' 的详细信息。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '小红书玻尿酸护肤 2024流行趋势 用户评价'}
[Tool Call] 模拟搜索网页：小红书玻尿酸护肤 2024流行趋势 用户评价
Observation: 工具返回结果：玻尿酸是2024年最热门的护肤成分。小红书话题#玻尿酸护肤有超100万浏览量。用户反馈：补水能力强、适合所有肤质、与其他成分配合效果更佳。常见浓度为0.1%~2%。
-- Iteration 3 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '玻尿酸护肤卖点 使用方法 注意事项 小红书爆款'}
[Tool Call] 模拟搜索网页：玻尿酸护肤卖点 使用方法 注意事项 小红书爆款
Observation: 工具返回结果：玻尿酸是2024年最热门的护肤成分。小红书话题#玻尿酸护肤有超100万浏览量。用户反馈：补水能力强、适合所有肤质、与其他成分配合效果更佳。常见浓度为0.1%~2%。
-- Iteration 4 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '玻尿酸补水保湿护肤惊喜效果'}
[Tool Call] 模拟生成表情符号，上下文：玻尿酸补水保湿护肤惊喜效果
Observation: 工具返回结果：['💦', '💧', '🌊', '✨', '💙']
-- Iteration 5 --
[模型生成结果] **Observation**: 我获得了适合玻尿酸补水主题的表情符号：💦、💧、🌊、✨、💙。现在我可以基于收集到的信息创作一篇活泼搞怪的玻尿酸小红

In [13]:
# 测试案例 3: 手部护理霜
product_name_3 = "手部护理霜"
tone_style_3 = "活泼甜美"
result_3 = generate_rednote(product_name_3, tone_style_3)

print("\n--- 生成的文案 3 ---")
print(result_3)


🚀 启动小红书文案生成助手，产品：手部护理霜，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '2024手部护理霜 小红书爆款 流行趋势'}
[Tool Call] 模拟搜索网页：2024手部护理霜 小红书爆款 流行趋势
Observation: 工具返回结果：手部护理霜销量暴增，消费者将其视为'第二张脸'。产品特点：快速吸收、持久保湿、改善手部暗沉和干纹。早晚使用效果最佳，按摩可促进吸收。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '手部护理霜'}
[Tool Call] 模拟查询产品数据库：手部护理霜
Observation: 工具返回结果：手部护理霜：富含甘油、角鲨烷、维E和玫瑰果油。功效：深层滋养、快速吸收、改善手部干纹和暗沉、柔软嫩白。质地丰润但不油腻，独特香气。规格：75ml。价格：¥89/罐。早晚涂抹配合按摩，1周见效。
-- Iteration 3 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '手部护理 嫩白保湿 惊喜效果 甜美可爱'}
[Tool Call] 模拟生成表情符号，上下文：手部护理 嫩白保湿 惊喜效果 甜美可爱
Observation: 工具返回结果：['💦', '💧', '🌊', '✨', '💙']
-- Iteration 4 --
[模型生成结果] **Observation**: 我已经收集到了足够的信息：
1. 手部护理霜被视为"第二张脸"护理，销量暴增
2. 产品含有甘油、角鲨烷、维E和玫瑰果油等成分
3. 功效包括深层滋养、改善干纹暗沉、柔软嫩白
4. 质地丰润不油腻，有独特香气
5. 早晚使用配合按摩，1周见效

现在我将创作一篇活泼甜美的小红书爆款文案：

```json
{
  "title": "救命！这手部护理霜让我拥有漫画少女手✨一周逆袭白嫩手",
  "body": "姐妹们！我终于找到了我的本命手部护理霜💖\n\n之前手部总是干

### 格式化 小红书文案

**格式化函数 `format_rednote_for_markdown` 的功能：**

1. 解析 JSON 字符串。
2. 提取标题、正文、标签和表情符号。
3. 将它们组合成一个易读的 Markdown 格式的文本。


**工作方式：**

1. **解析 JSON**：使用 `json.loads()` 将输入的字符串转换为 Python 字典。如果解析失败，会返回一个错误信息。
2. **提取数据**：使用 `.get()` 方法从字典中安全地提取 `title`、`body` 和 `hashtags`。使用 `.get()` 的好处是，如果某个键不存在，它会返回一个默认值（例如 `None` 或空列表），而不是抛出 `KeyError`。
3. **构建 Markdown 标题**：将 `title` 格式化为 Markdown 的二级标题 (`## Title`)。
4. **处理正文**：直接使用 `body`。由于小红书正文中的换行很重要，我们保留它们。
5. **处理 Hashtags**：将 `hashtags` 列表中的每个标签用空格连接起来，形成一行。
6. **表情符号 (Emojis)**：在小红书的实际发布中，表情符号通常已经嵌入在标题和正文中了。这个函数没有单独列出它们，因为这通常不是最终发布格式的一部分。如果需要，可以取消注释相关代码来单独显示它们。
7. **返回结果**：返回拼接好的 Markdown 字符串，并使用 `.strip()` 去除可能存在于末尾的多余空白。

In [14]:
import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [15]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟",
  "body": "姐妹们！我终于找到了我的本命美白精华！💖\\n\\n作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩\\n\\n🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！\\n💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！\\n🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！\\n\\n使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～\\n\\n真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖",
  "hashtags": ["#美白精华", "#提亮肤色", "#淡化痘印", "#护肤好物", "#冷白皮"],
  "emojis": ["✨", "💖", "🤩", "💧", "🌿"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = "{'title': 'Test', 'body': 'This is not valid json'}" # 使用单引号，非法
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)


--- 格式化后的小红书文案 (Markdown) ---
## ✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟

姐妹们！我终于找到了我的本命美白精华！💖

作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩

🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！
💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！
🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！

使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～

真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖

#美白精华 #提亮肤色 #淡化痘印 #护肤好物 #冷白皮

--- 格式化错误示例 ---
错误：无法解析 JSON 字符串 - Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
原始字符串：
{'title': 'Test', 'body': 'This is not valid json'}


In [17]:
# 调用格式化函数
markdown_note_1 = format_rednote_for_markdown(result_1)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note_1)

--- 格式化后的小红书文案 (Markdown) ---
## 2024小红书美妆趋势全解析✨温柔知性风正当时

姐妹们好呀～今天想和大家聊聊2024年小红书上最火的美妆趋势，用温柔知性的视角带你们看懂这些流行密码✨

💫【护肤趋势】
1. '早C晚A'理念持续火热🔥
   早上用维C抗氧化，晚上用A醇抗老修复，科学护肤才是王道

2. 屏障修复成为新焦点💧
   敏感肌友好产品大受欢迎，温和修复才是长久之计

3. 成分党依然活跃👍
   大家越来越关注产品成分，理性消费成为主流

💄【彩妆趋势】
1. '伪素颜'妆容持续走红✨
   自然通透的底妆，若有似无的眼妆，打造天生好皮感

2. 氛围感妆容成为新宠🎉
   重点不在技术多高超，而在整体氛围的营造

3. 多巴胺色彩点缀生活💖
   亮色系彩妆作为点缀，为日常妆容增添活力

📝【我的温柔建议】
1. 不要盲目跟风，选择适合自己的最重要
2. 护肤要循序渐进，给皮肤适应的时间
3. 彩妆是表达自我的方式，享受过程最重要

希望这篇趋势分析能给你们带来启发～记得找到最适合自己的美妆方式哦💕

#小红书美妆趋势 #2024美妆流行 #温柔知性风 #护肤心得 #彩妆技巧


In [18]:
# 调用格式化函数
markdown_note_2 = format_rednote_for_markdown(result_2)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note_2)

--- 格式化后的小红书文案 (Markdown) ---
## 救命！我的脸终于喝饱水了💦玻尿酸让我从沙漠变绿洲！

姐妹们！我宣布玻尿酸就是我2024年的本命护肤成分！🥳

之前我的脸干到能当砂纸用，上妆卡粉卡到怀疑人生😭直到我遇到了玻尿酸这个补水小天才！

✨使用体验：
1. 质地像水一样清爽，完全不黏腻
2. 吸收超快，涂完感觉脸在咕咚咕咚喝水
3. 第二天起床脸嫩得像剥壳鸡蛋🥚

🤣搞笑时刻：
男朋友说我最近脸太嫩，摸起来像在摸婴儿屁股👶
同事问我是不是偷偷去做了水光针💉

💡使用小贴士：
• 洁面后第一步使用，锁水效果加倍
• 可以混在面霜里，保湿力up up
• 妆前用一点，底妆服帖到哭

⚠️注意：玻尿酸浓度0.1%-2%最合适，太高反而会倒吸水分哦！

现在我的脸水润到能反光✨沙漠皮姐妹快冲！这波入股不亏！

#玻尿酸护肤 #补水神器 #沙漠皮救星 #护肤日常 #好物分享


In [19]:
# 调用格式化函数
markdown_note_3 = format_rednote_for_markdown(result_3)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note_3)

--- 格式化后的小红书文案 (Markdown) ---
## 救命！这手部护理霜让我拥有漫画少女手✨一周逆袭白嫩手

姐妹们！我终于找到了我的本命手部护理霜💖

之前手部总是干巴巴的，关节处还有暗沉，拍照都不敢露手😭
直到闺蜜推荐了这个宝藏手部护理霜，真的绝了！

✨使用感受：
质地像冰淇淋一样丝滑，一抹就化开💦
吸收超快，完全不油腻，涂完就能玩手机📱
淡淡的玫瑰果香超好闻，每次涂都像在做SPA🌹

✨成分亮点：
• 甘油+角鲨烷：深层锁水保湿
• 维E+玫瑰果油：改善暗沉提亮肤色
• 天然植物精华：温和不刺激

✨使用效果：
坚持用了一周，手部变化太明显了！
✅干纹细纹明显减少
✅关节暗沉改善很多
✅皮肤变得又软又嫩
✅自带光泽感，像打了高光✨

✨使用方法：
早晚各一次，取适量在掌心搓热
从指尖到手腕仔细按摩，促进吸收
睡前可以厚涂当手膜，效果加倍💅

现在拍照都敢大大方方展示手部了
闺蜜都说我的手像漫画里走出来的少女手
75ml大容量才¥89，性价比绝了！

姐妹们，手是女人的第二张脸
一定要好好护理呀💕

#手部护理 #护手霜推荐 #少女手养成 #秋冬必备 #护肤好物

#手部护理 #护手霜推荐 #少女手养成 #秋冬必备 #护肤好物


## 6. 评估与优化

文案生成并非一蹴而就，需要持续的评估和优化。本节讨论一些评估方法和优化策略。

#### 评估文案质量：
*   **客观量化评估 (数据)：**
    *   **点赞/收藏/评论/分享：** 基础互动
    *   **曝光/阅读/点击/涨粉：：** 流量与曝光
    *   **停留时长/截图率：** 用户行为。
    *   **商品页浏览/加购/ROI/成交转化：** 商业价值
    *   **爆文率/同类横向对比：** 竞争对比
*   **主观内部评估 (人工)：**
    *   **相关性：** 是否符合产品特点和主题。
    *   **吸引力：** 标题是否抓人，内容是否流畅。
    *   **合规性：** 是否有敏感词、违规宣传。
    *   **风格匹配：** 是否符合小红书调性和指定语气。
    *   **用户画像：** 目标人群年龄、地域、兴趣标签。



#### 优化迭代方法：
*   **Prompt 调整：** 根据评估结果，精修 System Prompt、User Prompt，增加或修改 Few-shot 示例。
*   **工具扩充：** 引入新的工具（如敏感词检测工具、竞品分析工具）。
*   **RAG (检索增强生成)：** 结合更精准的内部知识库，减少幻觉。


## 7. 总结与展望

通过本次实战，我们成功构建了一个基于 DeepSeek Agent 的小红书爆款文案生成助手。我们学习了如何拆解需求、设计 Prompt、定义工具，并实现 Agent 的核心工作流。

Agent 在内容营销领域的潜力巨大，未来可以进一步拓展到：

*   **超个性化内容：** 根据用户数据，生成一对一的定制文案。
*   **多模态内容创作：** 结合图片、视频生成，实现图文音视频一体化。
*   **智能营销决策：** Agent 不仅生成内容，还能分析效果并给出投放建议。
*   **跨平台适配：** 快速生成适应不同社交媒体平台风格的文案。

同时，我们也需关注挑战，如确保内容真实性、处理高度主观情感、与现有工作流的无缝集成等。Agent 技术仍在快速发展，期待未来能带来更多惊喜！